In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.pyplot import imread
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D
from tensorflow.keras.utils import to_categorical

In [3]:
images = []
label_id = []
root_dir = "Train"
image_size = (50, 50)

for i in range(43):
    label_dir = os.path.join(root_dir, str(i))
    image_files = os.listdir(label_dir)
    for image_file in image_files:
        image_path = os.path.join(label_dir, image_file)
        img = Image.open(image_path)
        img = img.resize(image_size)
        img = np.array(img)
        images.append(img)
        label_id.append(i)

In [4]:
X = np.array(images)
Y = np.array(label_id)
X = X / 255
Y = to_categorical(Y)
X.shape, Y.shape

((39209, 50, 50, 3), (39209, 43))

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=4242)

In [6]:
X_train.shape, y_train.shape

((31367, 50, 50, 3), (31367, 43))

In [7]:
model = Sequential([
    Conv2D(64, (3,3), input_shape=(50, 50, 3), activation="relu", padding="same"),
    MaxPool2D((2,2)),
    Dropout(0.5),
    Conv2D(64, (3,3), activation="relu"),
    MaxPool2D((2,2)),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(43, activation="softmax")
])

In [8]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 50, 50, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 25, 25, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 11, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 11, 11, 64)        0

In [10]:
model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_valid, y_valid))

Epoch 1/5


2023-05-13 17:06:51.125242: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


246/246 [==============================] - 34s 134ms/step - loss: 2.4327 - accuracy: 0.3288 - val_loss: 1.1888 - val_accuracy: 0.6826
Epoch 2/5
246/246 [==============================] - 34s 137ms/step - loss: 1.1869 - accuracy: 0.6247 - val_loss: 0.5618 - val_accuracy: 0.8769
Epoch 3/5
246/246 [==============================] - 36s 145ms/step - loss: 0.8114 - accuracy: 0.7356 - val_loss: 0.3136 - val_accuracy: 0.9294
Epoch 4/5
246/246 [==============================] - 36s 145ms/step - loss: 0.6402 - accuracy: 0.7880 - val_loss: 0.2492 - val_accuracy: 0.9589
Epoch 5/5
246/246 [==============================] - 36s 147ms/step - loss: 0.5445 - accuracy: 0.8219 - val_loss: 0.1779 - val_accuracy: 0.9648


In [11]:
test_images = []
root_dir = "Test"
image_size = (50, 50)

image_files = os.listdir(root_dir)
label_dir = os.path.join(root_dir)
for image_file in image_files:
    image_path = os.path.join(label_dir, image_file)
    img = Image.open(image_path)
    img = img.resize(image_size)
    img = np.array(img)
    test_images.append(img)

In [49]:
print(image_path)

Test/07491.png


In [12]:
test = np.array(test_images)
test = test / 255
test.shape

(12630, 50, 50, 3)

In [29]:
test_df = pd.read_csv("Test.csv")
y_test = test_df["ClassId"].values

In [30]:
y_test = y_test[:1000]
y_test.shape

(1000,)

In [31]:
test_images = test[:1000]

In [57]:
y_pred = model.predict(test_images)
y_pred = np.argmax(y_pred, axis=1)
y_pred

32/32 [==============================] - 0s 9ms/step


array([ 8, 14,  1,  1,  8, 36, 10, 13, 18, 18,  2,  5,  4, 25,  5,  1, 13,
       15,  5,  3,  3,  6,  2,  4,  9,  3,  3, 26, 23,  7, 18,  1, 10, 20,
        2, 31,  1, 23, 22, 10,  7,  9,  6, 25, 28, 12,  8,  9, 13,  7, 10,
        7, 18, 14,  8, 34,  7,  2,  1, 15,  4, 12, 25, 38,  1, 40, 31, 10,
       38, 17,  4,  7,  4,  1, 38,  2, 26, 31,  7, 38, 23,  4,  4, 21, 41,
        5, 36, 17, 13, 38, 37, 31,  9,  2, 10, 11,  7, 12,  7,  7, 25,  2,
       13, 20,  2, 10,  7,  5,  2,  9,  2, 31, 10,  7,  8, 39, 13,  9, 11,
       11,  1,  4, 10, 17, 28,  5, 11, 38, 10,  5,  1, 33,  8,  7, 16, 23,
       10, 13, 10,  7, 12, 14,  8, 18, 25, 25, 25, 35, 13,  9, 14, 20,  5,
       38, 21,  8, 13, 38, 12, 38, 31,  8,  1, 23,  0, 38, 23,  4, 13, 38,
        1, 38,  6,  7, 14, 10, 39,  5, 10,  8, 33,  5, 14, 31,  9,  1,  9,
        5, 18, 31,  2,  6, 39, 38, 11, 12, 35, 12, 25,  5, 38, 26,  4,  9,
       20,  1, 12,  5, 35,  1,  7,  5, 38,  5,  1, 11, 33,  7, 29, 13, 12,
       42,  1,  9, 14, 21

In [58]:
classes = [ 'Speed limit (20km/h)',
            'Speed limit (30km/h)', 
            'Speed limit (50km/h)', 
            'Speed limit (60km/h)', 
            'Speed limit (70km/h)', 
            'Speed limit (80km/h)', 
            'End of speed limit (80km/h)', 
            'Speed limit (100km/h)', 
            'Speed limit (120km/h)', 
            'No passing', 
            'No passing veh over 3.5 tons', 
            'Right-of-way at intersection', 
            'Priority road', 
            'Yield', 
            'Stop', 
            'No vehicles', 
            'Veh > 3.5 tons prohibited', 
            'No entry', 
            'General caution', 
            'Dangerous curve left', 
            'Dangerous curve right', 
            'Double curve', 
            'Bumpy road', 
            'Slippery road', 
            'Road narrows on the right', 
            'Road work', 
            'Traffic signals', 
            'Pedestrians', 
            'Children crossing', 
            'Bicycles crossing', 
            'Beware of ice/snow',
            'Wild animals crossing', 
            'End speed + passing limits', 
            'Turn right ahead', 
            'Turn left ahead', 
            'Ahead only', 
            'Go straight or right', 
            'Go straight or left', 
            'Keep right', 
            'Keep left', 
            'Roundabout mandatory', 
            'End of no passing', 
            'End no passing veh > 3.5 tons' 
]

In [59]:
print("Original: ", classes[y_test[34]])
print("Predicted: ", classes[y_pred[34]])

Original:  Speed limit (50km/h)
Predicted:  Speed limit (50km/h)


In [61]:
count = 0
for i in range(1, 1000):
    if classes[y_pred[i]] == classes[y_test[i]]:
        count += 1
        
print(count/10 , "%")

4.3 %
